Transformer(Bert)

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 10.5 MB/s eta 0:00:00


In [2]:
import json
import pandas as pd
import random
import pyarrow as pa
import datasets
import tqdm
import string
tqdm.tqdm.pandas()

### preprocessing and EDA

In [3]:
import pandas as pd
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go

In [4]:
maindf=pd.read_csv('sentiment_dataset.csv')

Neutral Data is very low and no more data is available so i remove neutral data and work with positive and negetive sentiment

In [5]:
maindf=maindf[maindf.sentiment !=0]

In [6]:
maindf.sentiment.replace(-1.0,0,inplace=True)

Data is Imbalance so undersample to lower count data

In [7]:
from imblearn.under_sampling import RandomUnderSampler

In [8]:
rus = RandomUnderSampler(sampling_strategy='not minority', random_state=1)
df_balanced, balanced_labels = rus.fit_resample(maindf, maindf['sentiment'])

In [9]:
fig = px.bar(df_balanced['sentiment'].value_counts().rename_axis('احساس').reset_index(name='تعداد'), x="احساس", y="تعداد", color="تعداد")
fig.update_layout(title_text="توزیع داده احساس", title_x=0.5)
fig.show()

In [10]:
df_balanced.sentiment.value_counts()

0.0    34908
1.0    34908
Name: sentiment, dtype: int64

In [11]:
df_balanced['sentiment']=df_balanced['sentiment'].astype(int)

In [12]:
df_balanced.sentiment.value_counts()

0    34908
1    34908
Name: sentiment, dtype: int64

In [13]:
def clean_data(text):
  english_alphabet=list(string.ascii_letters)
  numbers=list('0123456789۰١۲۳۴۵۶۷۸۹')
  for char in english_alphabet+numbers:
    text=text.replace(char,"")
  text=text.replace("/","")
  return text

In [14]:
df_balanced['Text']=df_balanced['Text'].progress_apply(lambda x:clean_data(x))

100%|██████████| 69816/69816 [00:03<00:00, 20205.78it/s]


In [15]:
from sklearn.model_selection import train_test_split

In [16]:
train_d,test_d =train_test_split(df_balanced,shuffle=True,test_size=0.2,stratify=df_balanced['sentiment'])

In [17]:
test=[]
train=[]
for i in range(len(test_d)):
   td={'text':list(test_d['Text'])[i],'label':list(test_d['sentiment'])[i]}
   test.append(td)
for i in range(len(train_d)):
   td={'text':list(train_d['Text'])[i],'label':list(train_d['sentiment'])[i]}
   train.append(td)

In [18]:
train=pa.Table.from_arrays([[a['text'] for a in train],[a['label'] for  a in train]],names=['text','label'])
test=pa.Table.from_arrays([[a['text'] for a in test],[a['label'] for  a in test]],names=['text','label'])
datasetsDict={}
datasetsDict['train']=datasets.Dataset(train)
datasetsDict['test']=datasets.Dataset(test)
dts=datasets.DatasetDict(datasetsDict)

In [19]:
dts

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 55852
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 13964
    })
})

In [23]:
!pip install transformers[torch]
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.6 MB/s eta 0:00:00


In [24]:
import torch
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification,TrainingArguments,Trainer

In [25]:
def gpu_check():
  torch.__version__
  #If ther 's a GPU available...
  if torch.cuda.is_available():
    #Tell Pytorch to use GPU.
    device = torch.device("cuda")
    print("There are %d GPU(s) avalilable." % torch.cuda.device_count())
    print('We will use the GPU:',torch.cuda.get_device_name(0))
    #If not..
  else:
    print('No GPU available,using the CPU instead.')
    device = torch.device("cpu")
    return device

In [26]:
device=gpu_check()
print(device)

There are 1 GPU(s) avalilable.
We will use the GPU: Tesla T4
None


In [27]:
label2id={'negetive':0,'positive':1}
id2label={0:'negetive',1:'positive'}

In [28]:
modelname="HooshvareLab/bert-base-parsbert-uncased"
tokenizer=AutoTokenizer.from_pretrained(modelname,model_max_length=100, add_special_tokens = True)
model=AutoModelForSequenceClassification.from_pretrained(
    modelname,num_labels=2,id2label=id2label,label2id=label2id
).to(device)
data_collector = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
from sklearn.metrics import precision_recall_fscore_support,accuracy_score

In [30]:
def preprocess_function(examples):
  return tokenizer(examples["text"],truncation=True)
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, fscore, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy=accuracy_score(labels, preds)

    # Check if support is None and replace it with 0
    # if support is None:
    #     support = 0

    metrics = {
        'precision': precision,
        'recall': recall,
        'F1-score': fscore,
        # 'support': support,
        'accuracy': accuracy,
    }
    return metrics

In [31]:
tokenized_data=dts.map(preprocess_function,batched=True)

Map:   0%|          | 0/55852 [00:00<?, ? examples/s]

Map:   0%|          | 0/13964 [00:00<?, ? examples/s]

In [33]:
training_args = TrainingArguments(
    output_dir="Persian-Text-Sentiment-Bert-V1/",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy ="epoch",
    save_strategy ="epoch",
    load_best_model_at_end=True,
    push_to_hub=False
)

In [34]:
trainer =Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collector,
    compute_metrics=compute_metrics
)

In [35]:
print("شروع آموزش...")
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


شروع آموزش...


Epoch,Training Loss,Validation Loss,Precision,Recall,F1-score,Accuracy
1,0.309700,0.326462,0.872697,0.871598,0.871504,0.871598
2,0.268600,0.360209,0.878527,0.875824,0.875601,0.875824
3,0.213700,0.382845,0.875931,0.872386,0.872085,0.872386
4,0.182300,0.554518,0.863713,0.863649,0.863643,0.863649
5,0.134600,0.629476,0.857231,0.856631,0.856571,0.856631
6,0.100100,0.850085,0.860554,0.860427,0.860415,0.860427
7,0.071000,1.019235,0.859645,0.859424,0.859403,0.859424
8,0.060400,1.044935,0.855347,0.855342,0.855342,0.855342
9,0.031200,1.167652,0.859793,0.859782,0.859781,0.859782
10,0.022000,1.212758,0.859338,0.859138,0.859118,0.859138


TrainOutput(global_step=34910, training_loss=0.13964399332578117, metrics={'train_runtime': 8686.7456, 'train_samples_per_second': 64.296, 'train_steps_per_second': 4.019, 'total_flos': 1.87492424160756e+16, 'train_loss': 0.13964399332578117, 'epoch': 10.0})

In [36]:
trainer.evaluate()

{'eval_loss': 0.3264618515968323,
 'eval_precision': 0.8726965774059261,
 'eval_recall': 0.8715983958751075,
 'eval_F1-score': 0.8715037396396725,
 'eval_accuracy': 0.8715983958751075,
 'eval_runtime': 54.0062,
 'eval_samples_per_second': 258.563,
 'eval_steps_per_second': 16.165,
 'epoch': 10.0}

In [ ]:
trainer.save_model('sentiment')

In [ ]:
!zip -r emotion_bert_model.zip 'sentiment'

In [20]:
!pip install huggingface_hub

In [21]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cr

In [22]:
dts.push_to_hub('SeyedAli/Persian-Text-Sentiment')

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/56 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/14 [00:00<?, ?ba/s]

In [37]:
trainer.push_to_hub('SeyedAli/Persian-Text-Emotion-Bert-V1')

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/651M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

'https://huggingface.co/SeyedAli/Persian-Text-Sentiment-Bert-V1/tree/main/'